# Biopython-Entrez
##  Search

In [72]:
from Bio import Entrez
import json
from bs4 import BeautifulSoup as bs
import lxml
Entrez.email = "XXX@YYY.com"
handle = Entrez.esearch(db="pubmed", retmax=10, term="kruse eiken vestergaard", idtype="acc")
record = Entrez.read(handle)
handle.close()

In [43]:
def pretty_print_json(json_):
    print(json.dumps(json_, indent=4, sort_keys=True))

In [73]:
record['IdList']

['29569152', '28197643', '27848006', '26679436', '26659068', '26223424', '25466529']

## Summary

In [46]:
handle = Entrez.esummary(db="pubmed", id="29569152")
record = Entrez.read(handle)
pretty_print_json( record[0] )

{
    "ArticleIds": {
        "doi": "10.1007/s00198-018-4467-z",
        "eid": "29569152",
        "medline": [],
        "pii": "10.1007/s00198-018-4467-z",
        "pubmed": [
            "29569152"
        ],
        "rid": "29569152"
    },
    "AuthorList": [
        "Kruse C",
        "Goemaere S",
        "De Buyser S",
        "Lapauw B",
        "Eiken P",
        "Vestergaard P"
    ],
    "DOI": "10.1007/s00198-018-4467-z",
    "ELocationID": "doi: 10.1007/s00198-018-4467-z",
    "EPubDate": "2018 Mar 22",
    "ESSN": "1433-2965",
    "FullJournalName": "Osteoporosis international : a journal established as result of cooperation between the European Foundation for Osteoporosis and the National Osteoporosis Foundation of the USA",
    "HasAbstract": 1,
    "History": {
        "accepted": "2018/02/25 00:00",
        "entrez": "2018/03/24 06:00",
        "medline": [
            "2019/10/24 06:00"
        ],
        "pubmed": [
            "2018/03/24 06:00"
        ],
     

## Fetch

In [84]:
#uid = "29569152"
uid = record['IdList']
handle = Entrez.efetch(db="pubmed", id=uid, rettype='Medline', retmode='xml')
result = handle.readlines()
result = "".join(result)
bs_content = bs(result, "lxml")
handle.close()

Use the 'abstract' parent and from that, iterate over the abstracttext to one string:

In [116]:
len( bs_content.find_all('abstract') )

7

Exemplified by the first record:

In [115]:
bs_content.find_all('abstract')[0]

<abstract>
<abstracttext>There is an increasing awareness of sarcopenia in older people. We applied machine learning principles to predict mortality and incident immobility in older Belgian men through sarcopenia and frailty characteristics. Mortality could be predicted with good accuracy. Serum 25-hydroxyvitamin D and bone mineral density scores were the most important predictors.</abstracttext>
<abstracttext label="INTRODUCTION" nlmcategory="BACKGROUND">Machine learning principles were used to predict 5-year mortality and 3-year incident severe immobility in a population of older men by frailty and sarcopenia characteristics.</abstracttext>
<abstracttext label="METHODS" nlmcategory="METHODS">Using prospective data from 1997 on 264 older Belgian men (n = 152 predictors), 29 statistical models were developed and tuned on 75% of data points then validated on the remaining 25%. The model with the highest test area under the curve (AUC) was chosen as the best. From these, ranked predictor

Use '.text' to extract the actual value.
### TODO: Maybe extract the labels too?

In [114]:
bs_content.find_all('abstract')[0].find_all('abstracttext')[0].text

'There is an increasing awareness of sarcopenia in older people. We applied machine learning principles to predict mortality and incident immobility in older Belgian men through sarcopenia and frailty characteristics. Mortality could be predicted with good accuracy. Serum 25-hydroxyvitamin D and bone mineral density scores were the most important predictors.'

## Loop example: to create one abstract:

In [127]:
abstract = ''
for abstractText in bs_content.find_all('abstract')[0].find_all('abstracttext'):
    if abstractText.get('label') != None:
        abstract = abstract + " " + abstractText.get('label') + ": "
    abstract = abstract + abstractText.text
abstract

'There is an increasing awareness of sarcopenia in older people. We applied machine learning principles to predict mortality and incident immobility in older Belgian men through sarcopenia and frailty characteristics. Mortality could be predicted with good accuracy. Serum 25-hydroxyvitamin D and bone mineral density scores were the most important predictors. INTRODUCTION: Machine learning principles were used to predict 5-year mortality and 3-year incident severe immobility in a population of older men by frailty and sarcopenia characteristics. METHODS: Using prospective data from 1997 on 264 older Belgian men (n\u2009=\u2009152 predictors), 29 statistical models were developed and tuned on 75% of data points then validated on the remaining 25%. The model with the highest test area under the curve (AUC) was chosen as the best. From these, ranked predictor importance was extracted. RESULTS: Five-year mortality could be predicted with good accuracy (test AUC of .85 [.73; .97], sensitivit